In [1]:
import torch

In [2]:
vocab = torch.arange(20)
print(vocab)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])


In [20]:
words_starting_mask = torch.rand(*vocab.size()) < .3
# words_starting_mask = words_starting_mask
print(words_starting_mask)

tensor([False,  True, False, False, False, False,  True, False, False, False,
        False, False,  True, False, False, False, False,  True, False, False])


In [21]:
sent_batch = torch.randint(vocab.size(0), (3, 15))
print(sent_batch)

tensor([[ 4, 19, 13,  6, 12, 13,  2,  1, 16, 18, 11, 19,  3,  8, 14],
        [ 2,  2, 12,  1,  6, 13, 18,  3,  7, 13,  6, 13,  4,  4,  2],
        [12, 18,  1,  7, 19,  3, 18,  7, 12,  5, 17, 14,  2,  3,  9]])


In [40]:
mask_positions = (torch.rand(*sent_batch.size()) < .45)
print(mask_positions.long())

tensor([[0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]])


In [41]:
word_begins_mask = words_starting_mask.expand(sent_batch.size(0), -1).gather(1, sent_batch)

In [42]:
word_begins_mask

tensor([[False, False, False,  True,  True, False, False,  True, False, False,
         False, False, False, False, False],
        [False, False,  True,  True,  True, False, False, False, False, False,
          True, False, False, False, False],
        [ True, False,  True, False, False, False, False, False,  True, False,
          True, False, False, False, False]])

In [43]:
word_begins_mask = word_begins_mask & mask_positions

In [44]:
word_begins_mask

tensor([[False, False, False,  True,  True, False, False,  True, False, False,
         False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False],
        [False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False]])

In [51]:
new_mask = torch.full(word_begins_mask.size(), False).bool()
new_mask[:,:word_begins_mask.size(1) - 1] = word_begins_mask[:,1:]

In [53]:
mask_positions = mask_positions | new_mask

In [54]:
mask_positions

tensor([[False, False,  True,  True,  True,  True,  True,  True, False, False,
          True,  True,  True, False, False],
        [ True, False, False, False, False,  True,  True,  True,  True,  True,
          True, False, False, False, False],
        [False,  True,  True, False, False,  True, False,  True, False, False,
         False, False,  True, False,  True]])

In [65]:
vocab_size = 12008
vocab = torch.arange(vocab_size)
words_starting_mask = torch.rand(*vocab.size()) < .3

sent_batch = torch.randint(vocab.size(0), (128, 128))


word_begins_mask = words_starting_mask.expand(sent_batch.size(0), -1).gather(1, sent_batch)

def get_masks(sent_batch):
    mask_positions = (torch.rand(*sent_batch.size()) < .15)
    word_begins_mask = words_starting_mask.expand(sent_batch.size(0), -1).gather(1, sent_batch)
    new_mask = torch.full(word_begins_mask.size(), False).bool()
    new_mask[:,:word_begins_mask.size(1) - 1] = word_begins_mask[:,1:]
    
    return mask_positions | new_mask

In [66]:
%timeit get_masks(sent_batch)

174 µs ± 546 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
